<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.62
    Uninstalling yfinance-0.2.62:
      Successfully uninstalled yfinance-0.2.62
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-11 13:44:41
-------------------
qualified stocks: 90
with latest results: 29
still star stocks: 17
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  4.16 L
Current:  1.36 C
-------------------
Today PnL: 5.00 K (0.04%)
Current PnL: -13.96 L (-9.72%)
CY Booked + Current PnL: -7.87 L (-5.49%)
-------------------
Total profit:  3.95 L
Total loss:  -17.90 L
-------------------
Total Booked + Current PnL: 19.11 L (16.39%)
Total Booked PnL: 33.06 L (28.37%)
Curr Year Booked PnL: 6.08 L (4.48%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.05 C
Est FTT PnL: 69.47 L (51.17%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 10.81%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LICI,785.05,983.0,-2.96,H-LC,86.67,238105.0,43413.0,5691.0,1.34,22.30,2.39,25.21,34.0,7.63,1.81,31.66,XY25,NTT,INSURANCE
59,RELIANCE,1291.83,1526.0,6.87,X-LC,28.89,158769.0,17960.0,7557.0,1.26,12.75,4.76,18.13,19.0,2.38,1.21,25.36,XY25,NTT,REFINERIES
3,ADANIPORTS,1103.69,1583.0,3.24,M-LC,53.33,103362.0,25000.0,9034.0,-1.22,31.90,8.74,43.43,77.0,2.77,0.79,38.25,XY24,NTT,MISC
81,UJJIVANSFB,52.77,53.0,54.03,M-SC,82.22,133812.0,-8667.0,9287.0,-3.49,-6.08,6.94,0.44,241.0,-0.93,1.02,57.93,OX40N,NTT,BANKS
75,TATAELXSI,7332.28,7332.0,-11.78,X-MC,58.89,92568.0,-10084.0,10081.0,-0.27,-9.82,10.89,-0.00,55.0,-1.00,0.70,40.71,OX40N,NTT,IT


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,NATIONALUM,189.63,247.44,-36.43,L-SC,71.11,100658.0,154.0,30489.0,-0.93,0.15,30.29,30.49,246.0,0.01,0.76,32.61,X5K,ATH,METALS
20,COALINDIA,406.65,512.15,6.87,L-LC,32.22,149314.0,-1146.0,40180.0,1.00,-0.76,26.91,25.94,157.0,-0.03,1.13,13.93,XY25,ATH,MINING
33,HINDALCO,651.95,761.55,-12.54,M-LC,25.56,105248.0,936.0,16598.0,-0.08,0.90,15.77,16.81,71.0,0.06,0.80,17.09,X5K,ATH,METALS
30,HAVELLS,1588.02,2069.16,-4.62,X-LC,21.11,246574.0,-1157.0,76216.0,0.70,-0.47,30.91,30.30,50.0,-0.02,1.87,12.76,X40,ATH,ELECTRICAL
5,APOLLOHOSP,6904.43,8285.00,-16.33,H-LC,16.67,103395.0,-171.0,20875.0,-0.37,-0.17,20.19,20.00,32.0,-0.01,0.79,13.88,X40N,BTT,HEALTHCARE


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,VIPIND,488.80,489.00,-939.92,H-SC,88.89,80048.0,-15268.0,15305.0,10.84,-16.02,19.12,0.04,151.0,-1.00,0.61,57.88,OX40N,NTT,MISC
84,VALIANTORG,512.64,838.00,-395.70,H-SC,100.00,125749.0,-7537.0,92136.0,4.99,-5.66,73.27,63.47,144.0,-0.08,0.96,110.79,XR,NTT,CHEMICALS
63,SAMMAANCAP,170.35,326.00,-164.07,M-SC,55.56,78294.0,-23916.0,117308.0,3.56,-23.40,149.83,91.37,199.0,-0.20,0.59,27.10,XY25,NTT,FINANCE
31,HCLTECH,1647.60,1959.14,19.14,H-LC,35.56,104426.0,3922.0,15079.0,2.63,3.90,14.44,18.91,31.0,0.26,0.79,26.02,X40,ATH,IT
57,RAJESHEXPO,517.65,518.00,1912.63,M-SC,57.78,54911.0,-82266.0,82361.0,2.26,-59.97,149.99,0.07,233.0,-1.00,0.42,36.56,OX40N,NTT,JEWELLERY


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,UJJIVANSFB,52.77,53.00,54.03,M-SC,82.22,133812.0,-8667.0,9287.0,-3.49,-6.08,6.94,0.44,241.0,-0.93,1.02,57.93,OX40N,NTT,BANKS
0,5PAISA,564.75,565.00,152.40,H-SC,68.89,136389.0,-50543.0,50628.0,-3.13,-27.04,37.12,0.04,148.0,-1.00,1.04,30.48,OX40N,NTT,FINANCE
35,HINDZINC,514.80,744.74,39.88,M-LC,91.11,104580.0,1620.0,44363.0,-2.09,1.57,42.42,44.67,68.0,0.04,0.79,34.96,X5K,ATH,METALS
11,BANKINDIA,116.91,190.00,-28.22,M-MC,73.33,190281.0,10473.0,101934.0,-2.03,5.82,53.57,62.52,168.0,0.10,1.45,36.93,XR,NTT,BANKS
37,ICICIGI,1839.64,2260.25,-12.91,H-MC,47.78,143530.0,7397.0,23726.0,-1.93,5.43,16.53,22.86,33.0,0.31,1.09,18.73,X40,ATH,INSURANCE


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,UJJIVANSFB,52.77,53.0,54.03,M-SC,82.22,133812.0,-8667.0,9287.0,-3.49,-6.08,6.94,0.44,241.0,-0.93,1.02,57.93,OX40N,NTT,BANKS
73,SYMPHONY,1306.42,1306.0,-27.09,M-SC,3.33,155497.0,-15644.0,15596.0,0.19,-9.14,10.03,-0.03,189.0,-1.00,1.18,11.76,OX40N,NTT,DURABLES
75,TATAELXSI,7332.28,7332.0,-11.78,X-MC,58.89,92568.0,-10084.0,10081.0,-0.27,-9.82,10.89,-0.00,55.0,-1.00,0.70,40.71,OX40N,NTT,IT
86,VIPIND,488.80,489.0,-939.92,H-SC,88.89,80048.0,-15268.0,15305.0,10.84,-16.02,19.12,0.04,151.0,-1.00,0.61,57.88,OX40N,NTT,MISC
47,KANSAINER,299.63,340.0,-67.49,H-SC,7.78,226215.0,-43452.0,79786.0,-0.98,-16.11,35.27,13.47,143.0,-0.54,1.72,13.23,XY24,NTT,PAINTS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TCS,3794.03,4998.00,-16.80,X-LC,8.89,271198.0,-24736.0,118649.0,0.40,-8.36,43.75,31.73,1.0,-0.21,2.06,8.55,X40,BTT,IT
42,INFY,1461.46,2275.00,-9.55,X-LC,24.44,289765.0,28164.0,117471.0,1.43,10.77,40.54,55.67,2.0,0.24,2.20,17.53,X40,BTT,IT
51,LTIM,5564.16,7197.33,0.70,X-LC,61.11,193788.0,-6522.0,65307.0,-0.04,-3.26,33.70,29.35,3.0,-0.10,1.47,33.88,X200,ATH,IT
6,ASIANPAINT,2961.56,4250.00,-24.43,X-LC,6.67,150280.0,-51106.0,138723.0,-0.40,-25.38,92.31,43.51,5.0,-0.37,1.14,4.61,X40,BTT,PAINTS
34,HINDUNILVR,2413.81,2723.00,-14.85,X-LC,20.00,267288.0,-3059.0,37688.0,-0.36,-1.13,14.10,12.81,6.0,-0.08,2.03,10.62,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,UNIONBANK,123.87,163.0,-1.84,M-LC,87.78,172642.0,31802.0,12689.0,-1.60,22.58,7.35,31.59,88.0,2.51,1.31,49.97,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,3.24,M-LC,53.33,103362.0,25000.0,9034.0,-1.22,31.90,8.74,43.43,77.0,2.77,0.79,38.25,XY24,NTT,MISC
50,LICI,785.05,983.0,-2.96,H-LC,86.67,238105.0,43413.0,5691.0,1.34,22.30,2.39,25.21,34.0,7.63,1.81,31.66,XY25,NTT,INSURANCE


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,130.64,228.00,102.64,M-MC,75.56,230008.0,5046.0,162616.0,0.48,2.24,70.70,74.53,183.0,0.03,1.75,33.54,XY24,BTT,STEEL
35,HINDZINC,514.80,744.74,39.88,M-LC,91.11,104580.0,1620.0,44363.0,-2.09,1.57,42.42,44.67,68.0,0.04,0.79,34.96,X5K,ATH,METALS
17,CAMS,3643.00,5250.99,5.33,H-SC,81.11,118345.0,16341.0,28687.0,-0.81,16.02,24.24,44.14,125.0,0.57,0.90,34.86,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,3.24,M-LC,53.33,103362.0,25000.0,9034.0,-1.22,31.90,8.74,43.43,77.0,2.77,0.79,38.25,XY24,NTT,MISC
32,HEROMOTOCO,4311.81,6039.03,1.90,H-MC,67.78,155540.0,4627.0,55823.0,1.48,3.07,35.89,40.06,101.0,0.08,1.18,27.22,AR,ATH,AUTO


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,WHIRLPOOL,1167.49,2270.00,-32.64,M-SC,44.44,196421.0,24800.0,137259.0,1.37,14.45,69.88,94.43,191.0,0.18,1.49,44.97,XR,NTT,DURABLES
89,WIPRO,243.46,420.00,-6.08,M-LC,23.33,160865.0,9920.0,99527.0,1.80,6.57,61.87,72.51,70.0,0.10,1.22,11.17,XR,NTT,IT
11,BANKINDIA,116.91,190.00,-28.22,M-MC,73.33,190281.0,10473.0,101934.0,-2.03,5.82,53.57,62.52,168.0,0.10,1.45,36.93,XR,NTT,BANKS
32,HEROMOTOCO,4311.81,6039.03,1.90,H-MC,67.78,155540.0,4627.0,55823.0,1.48,3.07,35.89,40.06,101.0,0.08,1.18,27.22,AR,ATH,AUTO
39,INDIAMART,2327.09,4871.06,-49.41,H-SC,60.00,131705.0,8369.0,126463.0,-0.15,6.79,96.02,109.32,122.0,0.07,1.00,30.11,AR,ATH,MISC


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,59.14,80.00,-51.71,L-SC,93.33,70829.0,-42720.0,82771.0,1.82,-37.62,116.86,35.27,260.0,-0.52,0.54,73.36,XR,NTT,HOTELS
4,ALKYLAMINE,2347.98,4567.52,-7.21,X-SC,85.56,86430.0,-14533.0,109974.0,-0.18,-14.39,127.24,94.53,59.0,-0.13,0.66,28.14,SR,ATH,CHEMICALS
52,MASFIN,326.60,399.50,-20.51,H-SC,72.22,90480.0,-7500.0,29370.0,-0.08,-7.65,32.46,22.32,138.0,-0.26,0.69,31.23,XR,ATH,FINANCE
35,HINDZINC,514.80,744.74,39.88,M-LC,91.11,104580.0,1620.0,44363.0,-2.09,1.57,42.42,44.67,68.0,0.04,0.79,34.96,X5K,ATH,METALS
3,ADANIPORTS,1103.69,1583.00,3.24,M-LC,53.33,103362.0,25000.0,9034.0,-1.22,31.90,8.74,43.43,77.0,2.77,0.79,38.25,XY24,NTT,MISC


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VALIANTORG,512.64,838.00,-395.70,H-SC,100.00,125749.0,-7537.0,92136.0,4.99,-5.66,73.27,63.47,144.0,-0.08,0.96,110.79,XR,NTT,CHEMICALS
7,ASIANTILES,75.41,137.00,7055.56,M-SC,98.89,78496.0,-15314.0,91935.0,-0.52,-16.32,117.12,81.67,235.0,-0.17,0.60,52.01,XR,NTT,CERAMICS
46,JPPOWER,18.73,26.20,-32.62,L-SC,97.78,137147.0,-4695.0,61264.0,0.67,-3.31,44.67,39.88,256.0,-0.08,1.04,38.99,XY24,NTT,POWER
74,TANLA,917.30,1963.11,-33.79,H-SC,96.67,162564.0,-76851.0,349805.0,-1.00,-32.10,215.18,114.01,127.0,-0.22,1.24,49.10,AR,ATH,IT
78,TITAGARH,1097.23,1548.00,-6.89,H-SC,95.56,163468.0,-28547.0,107431.0,-1.05,-14.87,65.72,41.08,152.0,-0.27,1.24,38.35,XY24,NTT,ENGINEERING


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.88
1,25,43.76
2,50,72.59


In [21]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     32.31
XY25     15.50
XR       11.08
X40      11.02
OX40N     8.57
X40N      8.48
AR        8.05
X5K       2.35
X200      1.47
SR        1.17
Name: CurrAlloc%, dtype: float64

In [22]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    19.00
X-LC    18.29
M-SC    14.43
M-LC    10.21
H-MC     9.06
H-LC     7.89
X-MC     6.23
M-MC     5.87
L-SC     3.29
X-SC     2.83
L-LC     1.13
Name: CurrAlloc%, dtype: float64

In [23]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.23,-1.01,34.62
IT,11.25,-11.98,68.83
BANKS,8.65,-6.98,50.10
MISC,6.62,-24.11,72.64
FINANCE,6.46,-18.30,63.82
PAINTS,5.67,-16.85,39.88
INSURANCE,5.27,3.94,24.76
ELECTRICAL,5.19,-2.78,47.36
AUTO,4.79,-7.22,54.36


In [24]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2678383.0,25
XR,1103141.0,13
AR,1011779.0,9
X40,596778.0,8
XY25,491320.0,10
X40N,414163.0,8
OX40N,304110.0,11
SR,190850.0,2
X5K,91450.0,3


In [25]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1867773.0,18
M-SC,1426166.0,18
X-LC,805460.0,11
M-MC,572360.0,5
H-MC,534076.0,7
M-LC,372224.0,8
X-MC,324789.0,5
L-SC,295644.0,5
X-SC,261611.0,4


In [26]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,888191.0,6
M-SC,XY24,566673.0,5
H-SC,AR,530947.0,3
X-LC,X40,451059.0,4
M-MC,XY24,385114.0,3
M-SC,XR,340823.0,4
H-MC,XY24,219452.0,2
M-LC,XY24,199568.0,4
H-SC,XR,193715.0,3


In [27]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
